<a href="https://colab.research.google.com/github/paulodowd/EMATM0053_21_22/blob/main/SL4_NonVolatileMemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supplementary Labsheet 4: Non-Volatile Memory

This labsheet gives a brief overview of how to use the non-volatile memory on your 3Pi+.  Non-volatile means that you can save data to this memory, turn your 3Pi+ on and off again, and the data should still be saved.   This memory is called `EEPROM`, and we can use an Arduino Library `EEPROM.h` to read and write to it.

Note:
- The EEPROM is only guaranteed to have 100,000 write cycles.  Please avoid accidentally writing into your EEPROM continuously.
- It is possible to use `EEPROM.update()` rather than `EEPROM.write()`.  This only writes to EEPROM if the value has changed.  This will significantly increase the lifespan of the EEPROM memory.
- More information on using EEPROM can be found in the <a href="https://www.arduino.cc/en/Reference/EEPROM">Arduino Reference</a>.

# Example 1: Saving an 8x8 Map

The EEPROM natively stores bytes per cell of memory.  The memory is mapped linearly, meaning we can imagine the EEPROM as a 1-dimensional array. The address range goes from 0 to the upper limit of available memory.  For the 32u4, this is 1KB (1000 bytes).  

To store a 2D array of bytes, we can simply iterate through the 2D array and update a 1D index for each iteration:

```c
#include <EEPROM.h>

// Push buttons connected to:
#define BUTTON_A_PIN  14
#define BUTTON_B_PIN  30

// We will imagine this is data
// collected by the robot.
byte grid[8][8];

void setup() {
  

  // Use two push buttons to decide
  // whether to read or write when the
  // 3Pi+ is turned on.
  // Here, INPUT_PULLUP means that the 
  // pin will read HIGH by default, and
  // a button press will make it read LOW
  pinMode( BUTTON_A_PIN, INPUT_PULLUP );
  pinMode( BUTTON_B_PIN, INPUT_PULLUP );

  Serial.begin(9600);
  delay(1000);
  Serial.println("***RESET***");

  // Setup our grid in volatile memory
  // as if it had been data collected
  // by the robot.
  int x,y;
  for( y = 0; y < 8; y++ ) {
    for( x = 0; x < 8; x++ ) {

      // You might want to change this
      // to a new value so you can see
      // it working.
      grid[x][y] = 66;
    }
  }

  // Wait for a button press to decide 
  // what to do.
  int a;
  int b;
  a = HIGH;
  b = HIGH;
  do {
    Serial.println("A = write, B = read");
    a = digitalRead( BUTTON_A_PIN);
    b = digitalRead( BUTTON_B_PIN);
  } while( a == HIGH && b == HIGH ); 

  // Saves our volatile grid into EEPROM.
  if( a == LOW ) {
    Serial.println("Writing new values to EEPROM");
    writeGridToEEPROM();
    while(1) {
      Serial.println("Write done, please reset");
      delay(1000);
    }
    
  } else {  // Recovers data from EEPROM.
    Serial.println("Reading old values from EEPROM");
  }
  
}

// writes a 2D grid into the 1D 
// EEPROM.  Note, readFromEEPROM
// uses a consistent method to 
// get the data back out.
void writeGridToEEPROM() {
  int x, y, address;

  address = 0;
  for( y = 0; y < 8; y++ ) {
    for( x = 0; x < 8; x++ ) {

       // Update will only write to the EEPROM
       // if the value has changed.  This should
       // help the EEPROM to stay working for 
       // longer.
       EEPROM.update( address, grid[x][y] );

       address++; // adds 1 to address
    }
  }
  
}

// Serial prints the contents of 
// EEPROM
void readFromEEPROM() {
  int x, y, address;

  address = 0;
  for( y = 0; y < 8; y++ ) {
    for( x = 0; x < 8; x++ ) {

       // Get a bye at address from EEPROM
       byte value = EEPROM.read( address );

       // Print as output
       Serial.print( value );
       Serial.print(",");
       address++; // adds 1 to address
    }
    // Newline after print a row
    Serial.print("\n");
  }
  //Done.
  Serial.println("***********\n\n");
}

void loop() {
  // put your main code here, to run repeatedly:
  readFromEEPROM();
  delay(1000);
}

```

# Example 2: Saving data at a time interval

In this example, a `millis()` call is used to only update the EEPROM with a new value every 1 second.  The address to write to the EEPROM is advanced every second. Once the address has advanced to 120, or 120milliseconds, 2 minutes, the code becomes stuck in a while loop to end the procedure.

```c
#include <EEPROM.h>

// A timestamp to update the 
// eeprom with new data every
// second.
#define UPDATE_MS   1000      // period
unsigned long eep_update_ts;  // timestamp
int eeprom_address;

void setup() {
  // put your setup code here, to run once:
  Serial.begin(9600);
  delay(1000);
  Serial.println("***RESET***");

  // Start recording data from eeprom
  // address 0.
  eeprom_address = 0;
  
}

// put your main code here, to run repeatedly:
void loop() {

  // This if statement runs every UPDATE_MS
  // which was set to 1000ms (1s)
  if( millis() - eep_update_ts > UPDATE_MS ) {
    eep_update_ts = millis();

    // Save "99" to current eeprom address
    // Could be a different value.
    EEPROM.update( eeprom_address, 99);

    // Advance address for next time.
    eeprom_address++;
  }

  // 120seconds, or 2 minutes.
  if( eeprom_address > 120 ) {
    
    // An infinite while loop to stop
    // the robot.
    while(1) {
      // zero motor pwm.
      // indicate finished.
    }
  }

}
```